In [61]:
import numpy as np 
import itertools 
np.random.seed(0)

class Parameters:

    @staticmethod
    def generate_true_dist_parameters(len_x, lb, ub):
        elements = [0,1]
        dict_parameters = {"x1": {(): np.random.uniform(lb,ub)}}
        for k in range(2,len_x+1):
            permutations = list(itertools.product(elements, repeat=k-1))
            dict_parameters["x"+str(k)] = {perm: np.random.uniform(lb,ub) for perm in permutations}
        return dict_parameters
    
    @staticmethod
    def generate_logsitc_parameters(len_x, lb, ub):
        elements = [0,1]
        dict_parameters = {"x1":{"bias": np.random.uniform(lb, ub)}}

        for k in range(2, len_x+1):
            dict_parameters["x"+str(k)] = {"bias": np.random.uniform(lb,ub)}
            for j in range(1,k):
                dict_parameters["x"+str(k)]["x"+str(j)]= np.random.uniform(lb,ub)
        return dict_parameters
    
        

#Number of paramerters of the True distribution 

$$
p(x_1, x_2, x_3, \dots, x_5) = p(x_1) \cdot p(x_2 \mid x_1) \cdot p(x_3 \mid x_1, x_2) \cdots p(x_n \mid x_1, x_2, \dots, x_{4})
$$

#Total number of parameters x can take [0,1]

The total number of parameters is:

$$
1 + 2 + 2^2 + 2^3 + 2^4 = 31
$$


In [62]:
Parameters().generate_true_dist_parameters(len_x=5, lb = 0, ub = 1)


{'x1': {(): 0.5488135039273248},
 'x2': {(0,): 0.7151893663724195, (1,): 0.6027633760716439},
 'x3': {(0, 0): 0.5448831829968969,
  (0, 1): 0.4236547993389047,
  (1, 0): 0.6458941130666561,
  (1, 1): 0.4375872112626925},
 'x4': {(0, 0, 0): 0.8917730007820798,
  (0, 0, 1): 0.9636627605010293,
  (0, 1, 0): 0.3834415188257777,
  (0, 1, 1): 0.7917250380826646,
  (1, 0, 0): 0.5288949197529045,
  (1, 0, 1): 0.5680445610939323,
  (1, 1, 0): 0.925596638292661,
  (1, 1, 1): 0.07103605819788694},
 'x5': {(0, 0, 0, 0): 0.08712929970154071,
  (0, 0, 0, 1): 0.02021839744032572,
  (0, 0, 1, 0): 0.832619845547938,
  (0, 0, 1, 1): 0.7781567509498505,
  (0, 1, 0, 0): 0.8700121482468192,
  (0, 1, 0, 1): 0.978618342232764,
  (0, 1, 1, 0): 0.7991585642167236,
  (0, 1, 1, 1): 0.46147936225293185,
  (1, 0, 0, 0): 0.7805291762864555,
  (1, 0, 0, 1): 0.11827442586893322,
  (1, 0, 1, 0): 0.6399210213275238,
  (1, 0, 1, 1): 0.1433532874090464,
  (1, 1, 0, 0): 0.9446689170495839,
  (1, 1, 0, 1): 0.52184832175007

#Number of paramerters of the logistic approximation of the conditional probability 

$$
p(x_1, x_2, x_3, \dots, x_5) = p(x_1) \cdot p(x_2 \mid x_1) \cdot p(x_3 \mid x_1, x_2) \cdots p(x_n \mid x_1, x_2, \dots, x_{4})
$$

#Total number of parameters x can take [0,1]
$$
p(x_1, x_2, x_3, \dots, x_5) \approx \sigma(w_1^T x_1 + b_1) \cdot \sigma(w_2^T x_2 + b_2) \cdot \sigma(w_3^T x_3 + b_3) \cdots \sigma(w_5^T x_5 + b_5)
$$

Where:
$$
\sigma(z) = \frac{1}{1 + e^{-z}}
$$

The total number of parameters is:

$$
1 + 2 + 3 + 4 + 5 = 15
$$


In [63]:
#generating logistic parameters
parameter = Parameters().generate_logsitc_parameters(len_x=5, lb = 0, ub = 1)
#lets assume that this is the distribution we are planning acheieve and this is our curret approx, true distribtion which redcues our parameters 
parameter

{'x1': {'bias': 0.7742336894342167},
 'x2': {'bias': 0.45615033221654855, 'x1': 0.5684339488686485},
 'x3': {'bias': 0.018789800436355142,
  'x1': 0.6176354970758771,
  'x2': 0.6120957227224214},
 'x4': {'bias': 0.6169339968747569,
  'x1': 0.9437480785146242,
  'x2': 0.6818202991034834,
  'x3': 0.359507900573786},
 'x5': {'bias': 0.43703195379934145,
  'x1': 0.6976311959272649,
  'x2': 0.06022547162926983,
  'x3': 0.6667667154456677,
  'x4': 0.6706378696181594}}

True Distribution provides an exact representation but becomes computationally infeasible as \( n \) grows due to exponential parameter growth.

Logistic Approximation offers a more scalable approach, reducing the parameter count significantly while maintaining sufficient flexibility.

These methods illustrate the trade-off between accuracy and efficiency in probabilistic modeling.


# Generating 1 sample

In [64]:
#Geneating a sample from the distribution 

def sigmoid(z):
    return 1/(1+np.exp(-z))

def get_a_sample(parameters):

    x_sample = []
    p_chain = []
    prob = 1.0

    for i in range(len(parameters)):
        z = parameters["x"+str(i+1)]["bias"]
        for k in range(1,i+1):
            z += x_sample[k-1]*parameters["x"+str(i+1)]["x"+str(k)]
        p_est = sigmoid(z)

        s_i = np.random.binomial(1,p_est)
        x_sample += [s_i]
        prob *= p_est
        p_chain += [float(p_est)]
    return x_sample, prob, p_chain
    

sample, probability , chain = get_a_sample(parameters=parameter)

print(f'Sample : {sample}, Sample probability : {probability}, ')
print(f'Chain probability : {chain}')

Sample : [1, 1, 1, 1, 1], Sample probability : 0.33752692635670706, 
Chain probability : [0.6844360133921935, 0.7358645994418772, 0.7770437377025466, 0.9309908455336089, 0.9263749132521876]


# Generating n Samples


In [65]:
#generating n samples 

def generate_n_samples(num_samples, parameters):
    n_samples = []
    for num in range(num_samples):
        sample, _, _ = get_a_sample(parameters)
        n_samples += [sample]
    return n_samples
    

generate_n_samples(num_samples=10, parameters = parameter)



[[1, 0, 1, 1, 1],
 [1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1],
 [0, 1, 0, 1, 0],
 [1, 0, 1, 1, 1],
 [1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1],
 [1, 1, 0, 1, 1]]

# Training the Model

In [102]:
def initial_derivative(len_samples):
    dict_parameters = {}
    dict_parameters = {"x1":{"bias": 0}}
    for k in range(2, len_samples+1):
            dict_parameters["x"+str(k)] = {"bias": 0}
            for j in range(1,k):
                dict_parameters["x"+str(k)]["x"+str(j)]= 0

    return dict_parameters  

init_derivative_value  = initial_derivative(len_samples=5)
init_derivative_value

{'x1': {'bias': 0},
 'x2': {'bias': 0, 'x1': 0},
 'x3': {'bias': 0, 'x1': 0, 'x2': 0},
 'x4': {'bias': 0, 'x1': 0, 'x2': 0, 'x3': 0},
 'x5': {'bias': 0, 'x1': 0, 'x2': 0, 'x3': 0, 'x4': 0}}

In [112]:
for key,value in init_derivative_value.items():
    print(key,value)

x1 {'bias': 0}
x2 {'bias': 0, 'x1': 0}
x3 {'bias': 0, 'x1': 0, 'x2': 0}
x4 {'bias': 0, 'x1': 0, 'x2': 0, 'x3': 0}
x5 {'bias': 0, 'x1': 0, 'x2': 0, 'x3': 0, 'x4': 0}


defaultdict(None, {})